<a href="https://colab.research.google.com/github/jansoe/AICA/blob/main/TextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.9 MB/s eta 0:00:00


In [ ]:
import transformers
import torch
import textwrap 

import plotly.express as px

## Input : Tokens

In [ ]:
model_version = 'EleutherAI/gpt-neo-125m'
#model_version = 'dbmdz/german-gpt2'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_version)

In [ ]:
tokenizer.tokenize('This is a test of the tokenizer')

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġof', 'Ġthe', 'Ġtoken', 'izer']

In [ ]:
tokens = tokenizer('This is a test of the tokenizer')['input_ids']
tokens

[1212, 318, 257, 1332, 286, 262, 11241, 7509]

In [ ]:
[tokenizer.convert_ids_to_tokens(t) for t in tokens]

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġof', 'Ġthe', 'Ġtoken', 'izer']

In [ ]:
tokenizer.decode(tokens)

'This is a test of the tokenizer'

In [ ]:
tokenizer.tokenize('n0thing is corect')

['n', '0', 'thing', 'Ġis', 'Ġcore', 'ct']

In [ ]:
tokenizer.vocab

### Test of different tokenizer

Mögliche Modelle: 
* https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads
* https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads

In [ ]:
model_name = 'bert-base-uncased' #

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer.tokenize('!!! Test sentence !!!') 

['!', '!', '!', 'test', 'sentence', '!', '!', '!']

## Output

In [ ]:
model_version = 'EleutherAI/gpt-neo-125m'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_version)
model = transformers.AutoModelForCausalLM.from_pretrained(model_version)
#_ = model.to('cuda') # Modell auf GPU schieben

In [ ]:
prompt = "The sun is"
#prompt = "Is Peter a male or female name. Answer: male."


ids = tokenizer(prompt, return_tensors='pt')['input_ids']
ids

tensor([[ 464, 4252,  318]])

In [ ]:
with torch.no_grad():
    out = model.forward(ids)

In [ ]:
probabilities = torch.softmax(out.logits, 2).numpy().squeeze()

In [ ]:
top10 = probabilities[-1].argsort()[:-11:-1]
top10

array([22751,  7396,   262,   257,   503,   287,  4634,  1464,   319,
         991])

In [ ]:
probabilities[-1][top10]

array([0.13434376, 0.07112848, 0.02940449, 0.02640192, 0.02544004,
       0.02446919, 0.02341055, 0.02017728, 0.02001574, 0.01899583],
      dtype=float32)

In [ ]:
decoded_most_probable = [tokenizer.decode(i) for i in top10]

px.bar(
    x = decoded_most_probable, 
    y = probabilities[-1][top10], 
    title = 'Next token probability'
)

Der Textgenerierung liegt zugrunde, dass das Modell in jedem Schritt für jedes Token im Vokabular eine Wahrscheinlichkeit angibt, mit der das Modell das jeweilige Token für die nächste Position vorhersagt. Nun können wir 'greedy' einfach immer das wahrscheinlichste Token auswählen und zur nächsten Position schreiten. Aber dies führt oft zu durchwachsenen Resultaten (siehe https://arxiv.org/abs/1904.09751).

Für eine verbesserte Qualität der erzeugten Texte bieten sich verschiedene Einstellungsmöglichkeiten an:
- `repetition_penalty`: Tokens, die bereits generiert wurden, werden um diesen Faktor unwahrscheinlicher
- `do_sample`: Es wird nicht das wahrscheinlichste Token genommen sondern anhand der vorhergesagten Wahrscheinlichkeitsverteilung ein Token zufällig ausgewählt
  - `temperature`: Je höher die Temperatur, desto höher die Chance, dass auch unwahrscheinliche Tokens ausgewählt werden
  - `top_`: Einschränkungen für die zur Verfügung stehenden Tokens:
    - `top_k`: es werden nur die wahrscheinlichsten k Tokens zugelassen
    - `top_p`: von den wahrscheinlichsten Tokens werden so viele zugelassen, dass deren Gesamtwahrscheinlichkeit `p` nicht überschreitet.

Ausführlichere Informationen finden Sie auch [hier](https://towardsdatascience.com/how-to-sample-from-language-models-682bceb97277).

In [ ]:
prompt = 'The sun is'
torch.manual_seed(10) # the seed controls the randomness

output_sequences = model.generate(
    **tokenizer(prompt, return_tensors='pt'),
    max_length = 128, # Wieviele Token sollen maximal generiert werden
    repetition_penalty = None,
    do_sample = True,
    penalty_alpha = 0.9,
    temperature = 0.7,
    top_k = None, # set to None to ignore or 0 < k <= vocab_size
    top_p = None, # set to None to ignore or 0 < p <= 1
    pad_token_id = tokenizer.eos_token_id
)

In [ ]:
output_sequences

tensor([[  464,  4252,   318,   287,   262,  6766,     0,   198,   198,    40,
           550,   655,  5201,  3555,   257,  1492,   546,   262,   995,   447,
           247,    82,   717,  6591, 25872,    13,   632,   373,   281,  1593,
          2589,   329,   502,    11,   475,   314,   550,   284,  2245,   329,
           257,  2589,   290,  2074,   340,   257,  2589,   286,   616,  1204,
            13,  3244,   314,  7452,   262,  4252,    13,   198,   198,    40,
           550,   655,  5201,  3555,   262,   717,  6843,   286,   616,  1492,
           290,   314,   550,   281,  4998,   640,    13,   314,   373,   523,
          7867,   416,   340,    13,   314,  1807,   340,   373,  1223,   284,
           466,   351,   262,  4252,  7396,    11,  1223,   284,  2107,   351,
            11,  1223,   284,   466,   351,   262,  4252,   447,   247,    82,
          3386,    13,  1114,   502,    11,   428,   373,   262,   717,   640,
           314,   550,  1775,   262,  4252,  4485,  

In [ ]:
decoded = tokenizer.decode(output_sequences[0])
print(textwrap.fill(decoded, width = 80, replace_whitespace=False))

The sun is in the sky!

I had just finished reading a book about the world’s
first solar eclipse. It was an important moment for me, but I had to stop for a
moment and consider it a moment of my life. Then I faced the sun.

I had just
finished reading the first chapter of my book and I had an amazing time. I was
so inspired by it. I thought it was something to do with the sun rising,
something to live with, something to do with the sun’s forces. For me, this was
the first time I had seen the sun rise. I


### Aufgabe: Exploration der Parameter

Experimentieren Sie mit GPT-2 und den Modellfähigkeiten zur Textgenerierung. Wie können Sie die überzeugensten Texte generieren? Halten Sie Ihre Beobachtungen in geeigneter Weise hier im Notebook fest.

- Was beobachten Sie, wenn Sie `repetition_penalty = None` und `do_sample = False` setzten?
- Was passiert mit `do_sample = True` und verschiedenen Werten von `temperature` and `top_p`?
- Was passiert, wenn Sie mit `prompt_text = '<|endoftext|>'` startet?